In [52]:
# Import the Spotipy library for interacting with the Spotify Web API, allowing us to fetch and manage Spotify data.
import spotipy

# Import the utility module from Spotipy, providing helpful functions for authentication and token handling.
import spotipy.util as util

# Import the webbrowser module, enabling control over the web browser to open URLs (e.g., for authorization purposes).
import webbrowser

# Import the json library to handle JSON data (e.g., for reading/writing Spotify API responses).
import json

# Import the urllib.request module, which provides functions for opening URLs, handling HTTP requests, etc.
import urllib.request

# Import the NLTK (Natural Language Toolkit) stopwords, a list of common words (e.g., "the," "and") for text processing.
from nltk.corpus import stopwords

# Import NLTK's part-of-speech (POS) tagging tool, which labels words in a sentence with their parts of speech (e.g., noun, verb).
from nltk import pos_tag

# Import the requests library to make HTTP requests (alternative to urllib for handling API requests).
import requests

# Import the collections module, offering specialized data structures like dictionaries with default values, counters, etc.
import collections


In [53]:
# the variable key now contains my API key
with open("news_key.txt.txt", "r") as file:
    key = file.read()

In [88]:
# this is the url for the news api
url = f"https://newsapi.org/v2/top-headlines?category=technology&apiKey={key}"
url

'https://newsapi.org/v2/top-headlines?category=technology&apiKey=cda165020a74441fbe535a0d6ed3f805'

In [55]:
# request is when im asking for data
request = urllib.request.Request(url)
# response is the answer i get for asking for data
response = urllib.request.urlopen(request)

In [56]:
# i didn't remember what data type response is - so i printed it!
type(response)

http.client.HTTPResponse

In [57]:
# converting the HTTPResponse object to a python dictionary
headlines = json.loads(response.read())

In [58]:
headlines

{'status': 'ok',
 'totalResults': 31,
 'articles': [{'source': {'id': None, 'name': '9to5Mac'},
   'author': 'Filipe Espósito',
   'title': 'Apple announces Vision Pro international expansion in two more countries - 9to5Mac',
   'description': 'Following an exclusive launch in the US in February, Apple began the international expansion of Vision Pro in June. Now...',
   'url': 'https://9to5mac.com/2024/10/30/apple-vision-pro-international-expansion/',
   'urlToImage': 'https://i0.wp.com/9to5mac.com/wp-content/uploads/sites/6/2024/10/vision-pro-eyesight-2.jpeg?resize=1200%2C628&quality=82&strip=all&ssl=1',
   'publishedAt': '2024-10-31T03:14:00Z',
   'content': 'Following an exclusive launch in the US in February, Apple began the international expansion of Vision Pro in June. Now the company is ready to bring Apple Vision Pro to even more countries, as the c… [+1321 chars]'},
  {'source': {'id': None, 'name': 'KGW.com'},
   'author': 'Katherine Cook',
   'title': "Portland radio DJ surv

In [59]:
# make an empty list of my titles
# article_titles = []
# for headline in headlines['articles']:
#     article_titles.append(headline['title'])

article_titles = [x['title'] for x in headlines['articles']] # List Comprehension 

In [60]:
article_titles

['Apple announces Vision Pro international expansion in two more countries - 9to5Mac',
 "Portland radio DJ survives near-death experience and loses his job before finding a better life: 'I'm lucky' - KGW.com",
 'Call of Duty: Black Ops 6 sets record for "Game Pass subscriber adds on launch day", says Microsoft - Eurogamer',
 'Nintendo made a music streaming app for Switch Online subscribers - The Verge',
 'Google’s AI-powered weather app is rolling out to older Pixels - The Verge',
 'Red Dead Redemption Remastered is CPU-bound on PC, even with an AMD Ryzen 9 7950X3D - DSOGaming',
 'Android Trojan that intercepts voice calls to banks just got more stealthy - Ars Technica',
 "PlayStation Urgently Needs To Rethink Its Live-Service Strategy After Concord's Failure - IGN",
 'WordPress co-founder Matt Mullenweg says a fork would be ‘fantastic’ - TechCrunch',
 '10 Horror Video Games to Play for Halloween 2024 - Bloody Disgusting',
 'Google’s AI system could change the way we write: InkSight t

In [62]:
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\Tinu\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


True

In [77]:
# list of all possible POS tags from nltk: https://stackoverflow.com/questions/15388831/what-are-all-possible-pos-tags-of-nltk
list_of_words = []
tagged_words= pos_tag(headline[0].split())

tagged_words    # Position tag function- uses the natural langauage tokeniser and splits eveyrthing up foryou
proper_nouns = [word for word,pos in tagged_sent if pos == 'NNP'] #and word.lower() not in stopwords.words('english') and word.isalpha()])
proper_nouns


['Apple', 'Vision', 'Pro']

In [78]:
keywords = []
for article_title in article_titles:
    # tagged_words will return a list of tuples
    tagged_words = pos_tag(article_title.split())
    # collect all the proper nouns
    keywords.extend([word for word,tag in tagged_words if tag == 'NNP'])

keywords

['Apple',
 'Vision',
 'Pro',
 'Portland',
 'DJ',
 'Duty:',
 'Black',
 'Ops',
 'Pass',
 'subscriber',
 'Microsoft',
 'Nintendo',
 'Switch',
 'Online',
 'Google’s',
 'AI-powered',
 'Pixels',
 'Dead',
 'Redemption',
 'Remastered',
 'CPU-bound',
 'PC,',
 'AMD',
 'Ryzen',
 'Android',
 'Trojan',
 'Ars',
 'Technica',
 'PlayStation',
 'Live-Service',
 'Strategy',
 "Concord's",
 'Failure',
 'Matt',
 'Mullenweg',
 'Horror',
 'Video',
 'Games',
 'Halloween',
 'Google’s',
 'AI',
 'InkSight',
 "Here's",
 'Full',
 'Wizards',
 'Deck',
 'From',
 'Magic:',
 "Gathering's",
 'Foundations',
 "Beginner's",
 'Box',
 'New',
 'OnePlus',
 'Central',
 'Entire',
 'Mac',
 'Lineup',
 'Least',
 'RAM,',
 'Ending',
 'Era',
 'Death',
 'Note',
 'Series',
 'Gets',
 'New',
 'Online',
 'Social',
 'Deduction',
 'Game',
 'PS5,',
 'PS4,',
 'November',
 'Anime',
 'News',
 'Network',
 'Apple',
 'Refreshes',
 'MacBook',
 'Pro',
 'Laptops',
 'New',
 'M4',
 'Pro,',
 'M4',
 'Max',
 'Chips',
 'New',
 'Pokémon',
 'TCG',
 'Alexa',
 '

In [76]:
keywords=[]
my_lista=['Apple', 'Vision', 'Pro']
my_listb=['Sony', 'Playstation']
keywords.extend(my_lista)
keywords.extend(my_listb)

keywords

['Apple', 'Vision', 'Pro', 'Sony', 'Playstation']

In [80]:
word_counter = collections.Counter(keywords)
word_counter

Counter({'New': 4,
         'Apple': 2,
         'Pro': 2,
         'Online': 2,
         'Google’s': 2,
         'M4': 2,
         'Vision': 1,
         'Portland': 1,
         'DJ': 1,
         'Duty:': 1,
         'Black': 1,
         'Ops': 1,
         'Pass': 1,
         'subscriber': 1,
         'Microsoft': 1,
         'Nintendo': 1,
         'Switch': 1,
         'AI-powered': 1,
         'Pixels': 1,
         'Dead': 1,
         'Redemption': 1,
         'Remastered': 1,
         'CPU-bound': 1,
         'PC,': 1,
         'AMD': 1,
         'Ryzen': 1,
         'Android': 1,
         'Trojan': 1,
         'Ars': 1,
         'Technica': 1,
         'PlayStation': 1,
         'Live-Service': 1,
         'Strategy': 1,
         "Concord's": 1,
         'Failure': 1,
         'Matt': 1,
         'Mullenweg': 1,
         'Horror': 1,
         'Video': 1,
         'Games': 1,
         'Halloween': 1,
         'AI': 1,
         'InkSight': 1,
         "Here's": 1,
         'Full': 1

In [89]:
# if you want to remove duplicates in a list, convert to a dictionary and then back to a list
# because a dictionary does not allow duplicate keys and will remove them
list_of_words_no_repeat = list(dict.fromkeys(list_of_words))
list_of_words_no_repeat

[]

In [90]:
# open file with keys and set the path to your credentials JSON file
credentials = "spotify-keys.json"
with open(credentials, "r") as keys:
    api_tokens = json.load(keys)

In [91]:
client_id = api_tokens["client_id"]
client_secret = api_tokens["client_secret"]
redirectURI = api_tokens["redirect"]
username = api_tokens["username"]

In [92]:
scope = 'user-read-private user-read-playback-state user-modify-playback-state playlist-modify-public user-library-read'
token = util.prompt_for_user_token(username, scope, client_id=client_id,
                           client_secret=client_secret,
                           redirect_uri=redirectURI)

In [93]:
# print out token
token

'BQDgSAXsb4cEySczU2w956dEIElhEnP2x-DcoifThzYnmMYQ4f_N9SDLQC8HxMuo-eZ1wqLd4aL6iKiCsn0oc0EjqJ7R79BwwGkMO8T_h1mH04OxVZAy_USjVCysbnbZb7ApvdtuMwM9hGktuY7CWt-mua82X5qhU0fNsr9q1aoMWSLclsAiSnaGWFgePZ59hvNJz8GqGilHGq-9iPg3ynuYvzMqKK852Z2RhKP7YBnCxOcwBd95C1YuDjlvEf9PPj9d-aym5g'

In [96]:
# create my Spotify object
sp = spotipy.Spotify(auth=token)

In [97]:
# start a list of songs for my headline playlist
songs_for_playlist = []
for keyword in word_counter.keys():
    # only take the 'top' words
    if word_counter[keyword] > 1:
        # search for artists based on my keyword, only give me one artist
        searchResults = sp.search(q="artist:" + keyword, type="track", limit=1) # limit is how many songs per keyword
        # make sure that something gets returned
        if len(searchResults['tracks']['items']) > 0:
            songs_for_playlist.append(searchResults['tracks']['items'][0]['uri']) # Uniform resource identifier
    
print(songs_for_playlist)

['spotify:track:0IVkP59yJ9GFF6B7IrvrxA', 'spotify:track:67iAlVNDDdddxqSD2EZhFs', 'spotify:track:3ZXOkRlGr6qmiOb6FI8GwP', 'spotify:track:4Huu65kMNpBongpyBtPTDr', 'spotify:track:50SxWFCvmZCYcpKWPZECES', 'spotify:track:1H5PVxswRh4KpLoW5o78VJ']


In [98]:
my_playlist = sp.user_playlist_create(user=username, name="Todays News", public=True, #if you change to false it modifies the scope set earlier in the code 
                                      description="Songs for the news")
results = sp.user_playlist_add_tracks(username, my_playlist['id'], songs_for_playlist)
print(results)

{'snapshot_id': 'AAAAAptyXsTcUBKJ9kq6Vem12//aJetx'}


In [99]:
webbrowser.open(my_playlist['external_urls']['spotify'])


True